In [3]:
import pandas as pd
churn = pd.read_csv('/Users/changlu/Downloads/Churn_Modelling.csv')
churn

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import numpy as np

X = churn.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited'])
y = churn['Exited']


In [7]:
# Splitting the dataset into train and test sets again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Checking the shapes of the train and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 10), (2000, 10), (8000,), (2000,))

In [12]:
# Importing all necessary libraries and components
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd


# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Columns that need to be one-hot encoded
categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]

# Columns that need to be scaled
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(drop='first')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', rf_model)])
rf_pipeline.fit(X_train, y_train)

# Logistic Regression model
lr_model = LogisticRegression(max_iter=500, random_state=42)
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', lr_model)])
lr_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['CreditScore', 'Age',
                                                   'Tenure', 'Balance',
                                                   'NumOfProducts', 'HasCrCard',
                                                   'IsActiveMember',
                                                   'EstimatedSalary']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['Geography', 'Gender'])])),
                ('model', LogisticRegression(max_iter=500, random_state=42))])

In [13]:
rf_importance = rf_pipeline.named_steps['model'].feature_importances_

features_after_encoding = (rf_pipeline.named_steps['preprocessor']
                           .transformers_[1][1]
                           .get_feature_names_out(categorical_cols))
all_features = np.concatenate([numerical_cols, features_after_encoding])

lr_importance = lr_pipeline.named_steps['model'].coef_[0]

rf_importance, lr_importance, all_features


(array([0.14333835, 0.23692176, 0.08207956, 0.14161247, 0.13148586,
        0.01845421, 0.04072451, 0.14755811, 0.02619045, 0.01321387,
        0.01842083]),
 array([-0.06766121,  0.75417804, -0.04267046,  0.16139606, -0.06045392,
        -0.01023686, -0.53326673,  0.01578731,  0.77462491,  0.08911587,
        -0.53022349]),
 array(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
        'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
        'Geography_Germany', 'Geography_Spain', 'Gender_Male'],
       dtype=object))

In [24]:
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, 
                                                   feature_names=all_features, 
                                                   class_names=['Not Exited', 'Exited'], 
                                                   discretize_continuous=True)

i = np.random.randint(0, X_test.shape[0])

exp = explainer.explain_instance(X_test.iloc[i].values, 
                                 rf_pipeline.predict_proba, 
                                 num_features=4)

exp.show_in_notebook()



TypeError: unsupported operand type(s) for -: 'str' and 'str'